# WIC

The Special Supplemental Nutrition Program for Women, Infants, and Children (WIC) provides vouchers for food and formula to low-income mothers and children deemed to be at nutritional risk.
The Food and Nutrition Service (FNS), a division of the United States Department of Agriculture (USDA), administers WIC.

## Examples

Consider a family with a single mother and one child, both deemed to be at nutritional risk, and \$2,000 monthly income.
They will receive WIC vouchers until the child reaches age five, though their benefit will vary with the child's age and whether the mother breastfeeds.

In [1]:
from openfisca_us import IndividualSim
import numpy as np
import pandas as pd
import plotly.express as px


def wic_by_child_age(is_breastfeeding):

    sim = IndividualSim(year=2022)
    sim.add_person(name="child", is_wic_at_nutritional_risk=True)
    sim.add_person(
        name="mother",
        employment_income=2_000 * 12,
        # is_mother defaults to is_breastfeeding, but we need to set it
        # explicitly for non-breastfeeding mothers.
        is_mother=True,
        is_breastfeeding=is_breastfeeding,
        is_wic_at_nutritional_risk=True,
    )
    sim.add_family(members=["mother", "child"])
    sim.add_spm_unit(members=["mother", "child"])

    sim.vary("age", max=6, step=0.1)

    return pd.DataFrame(
        dict(
            is_breastfeeding=np.where(is_breastfeeding, "Yes", "No"),
            age=sim.calc("age")[0],
            wic=(sim.calc("wic").sum(axis=0) / 12).round(),
        )
    )


df = pd.concat([wic_by_child_age(True), wic_by_child_age(False)])

LABELS = dict(
    age="Child age",
    is_breastfeeding="Breastfeeding",
    wic="Monthly WIC benefit value",
)

fig = px.line(
    df,
    "age",
    "wic",
    color="is_breastfeeding",
    labels=LABELS,
    title="Total WIC benefit value for a single mother of one based on the child's age",
)
fig.update_layout(yaxis_tickformat="$,")
fig.show()